# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115775e+02     1.522756e+00
 * time: 0.47603321075439453
     1     1.046470e+01     8.188245e-01
 * time: 1.201470136642456
     2    -1.190917e+01     9.720182e-01
 * time: 1.2743380069732666
     3    -3.376998e+01     7.120621e-01
 * time: 1.3752670288085938
     4    -4.719957e+01     5.157074e-01
 * time: 1.4969241619110107
     5    -5.678709e+01     2.009511e-01
 * time: 1.579397201538086
     6    -5.981594e+01     1.136089e-01
 * time: 1.6404590606689453
     7    -6.095859e+01     5.490526e-02
 * time: 1.7024869918823242
     8    -6.140344e+01     4.273155e-02
 * time: 1.7734100818634033
     9    -6.164353e+01     2.894338e-02
 * time: 1.8560431003570557
    10    -6.187686e+01     2.272760e-02
 * time: 1.9179601669311523
    11    -6.202897e+01     1.927214e-02
 * time: 1.9789440631866455
    12    -6.209060e+01     1.478719e-02
 * time: 2.0397419929504395
    13    -6.214173e+01     1.066943e-02
 * time: 2.10097599029

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671063
    AtomicLocal         -18.8557690
    AtomicNonlocal      14.8522662
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336820

    total               -62.261666461590